In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用T5X模型的Model Garden Pipeline模板开始

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_template_pipelines_t5x.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_template_pipelines_t5x.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/communitymodel_garden/model_garden_template_pipelines_t5x.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本教程演示如何使用Vertex AI Pipelines修改、编译和执行预先构建的Vertex AI Model Garden管道模板。

了解有关[创建管道模板](https://cloud.google.com/vertex-ai/docs/pipelines/create-pipeline-template)的更多信息。

### 目标

在本教程中，您将学习如何使用`Vertex AI Pipelines`中的预构建流水线模板来微调一个 T5X 文本分类模型，其中模型是从`Vertex AI Model Garden`中访问的。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Pipelines`
- `Vertex AI Training`
- `Vertex AI Model Garden`
- `Google Cloud Pipeline Components`


执行的步骤包括：

- 在`Artifact Registry`中创建一个用户定义的存储库。
- 将预建流水线模板上传到`Artifact Registry`。
- 使用预建流水线模板创建一个流水线作业来微调一个 T5X 模型。
- 使用`Vertex AI Pipelines`执行流水线。
    - 从 Vertex AI Model Garden 加载 T5X 模型
    - 进行微调训练模型
- 从流水线工件获取 Vertex AI 模型资源。
- 将模型部署到 Vertex AI Endpoint
- 进行预测

### 模型

本教程使用了来自`Vertex AI Model Garden`的预训练T5文本分类模型，然后在一个数据集上进行微调（迁移学习），该数据集包含被分类为FirstClass或SecondClass的文本短语。

了解更多关于[文本到文本传输变换器](https://github.com/google-research/text-to-text-transfer-transformer)。

成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage
* Dataflow

了解有关 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage 价格](https://cloud.google.com/storage/pricing) 和 [Dataflow 价格](https://cloud.google.com/dataflow/pricing) 的信息，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装执行此笔记本所需的软件包。

*注意：* 此教程需要 KFP 2.x。

In [ ]:
import os

! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-pipeline-components \
                         kfp==2.0.0b15

### 仅限Colab：取消对以下单元格的注释以重启内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下步骤：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[找到项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 区域

您也可以更改Vertex AI使用的“REGION”变量。了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 启用API

您可以使用 `gcloud` 启用所需的API。

In [ ]:
! gcloud services enable compute.googleapis.com         \
                         containerregistry.googleapis.com  \
                         aiplatform.googleapis.com  \
                         artifactregistry.googleapis.com

### 认证您的Google Cloud账号

根据您的Jupyter环境，您可能需要手动认证。请按照以下相关说明进行操作。

1. Vertex AI 工作台
* 无需操作，因为您已经通过身份验证。

In [ ]:
# ! gcloud auth login

3.协作、取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务账号授予云存储权限。

### 创建一个云存储桶

创建一个存储桶来存储中间产品，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时：运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

服务账号

如果您不知道您的服务账号，请尝试使用`gcloud`命令在下面执行第二个单元格来获取您的服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

为 Vertex AI Pipelines 设置服务账号访问权限

运行以下命令，以授予您的服务账号对在前一步创建的存储桶中的管道工件进行读写操作的访问权限--您只需为每个服务账号运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator  
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer  

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import json
import os

import google.cloud.aiplatform as aiplatform
from kfp.registry import RegistryClient

### 初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

启用Artifact Registry API
您必须为您的项目启用Artifact Registry API服务。

<a href="https://cloud.google.com/artifact-registry/docs/enable-service">了解如何启用服务</a>。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

## 在Artifact Registry中创建仓库

首先，在`Artifact Registry`中创建您自己定义的仓库。您可以使用此仓库来上传和检索您的流水线模板。

In [ ]:
REPO_NAME = "my-docker-repo-unique"

! gcloud artifacts repositories create {REPO_NAME} --location={REGION} --repository-format=KFP

### 上传管道模板

接下来，实例化一个客户端接口到 Artifact Registry。然后使用 `upload_pipeline()` 方法上传您的管道模板。

In [ ]:
T5X_YAML = "gs://cloud-samples-data/vertex-ai/dataset-management/datasets/t5_finetuning/pipeline.yaml"

! gsutil cp {T5X_YAML} pipeline.yaml

client = RegistryClient(
    host=f"https://{REGION}-kfp.pkg.dev/{PROJECT_ID}/quickstart-kfp-repo"
)

templateName, versionName = client.upload_pipeline(
    file_name="pipeline.yaml",
    tags=["v1", "latest"],
    extra_headers={
        "description": "This is a pipeline template for fine-tuning a T5 model."
    },
)

! rm pipeline.yaml

查看您注册表中的工件

接下来，使用`gcloud artifacts files`命令，您可以查看工件，包括流水线模板，在您的工件存储库中。

In [ ]:
! gcloud artifacts files list  --repository={REPO_NAME} --location={REGION}

## 加载并执行管道作业

接下来，您可以通过实例化PipelineJob()从您的T5管道模板创建一个Vertex AI管道作业，以下是需要的参数：

- `display_name`: 管道作业的人类可读名称。
- `template_path`: 存储在Artifact Registry中的管道模板路径。
- `enable_caching`: 在重新运行时，使用以前成功且未更改的步骤的结果。
- `pipeline_root`: 用于存储管道结果的云存储位置。
- `parameter_values`: 输入到模板管道中的参数和值。在此示例中，它们是：
TODO
    - `project`: 你的项目ID。
    - `class_labels`: 一个按基数顺序排列的有效类标签列表。
    - `root_dir`: 一个云存储临时区域。
    - `training_data_path`: 训练数据的云存储位置。
    - `ground_truth_gcs_source_uris`: 评估数据的云存储位置。

In [ ]:
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/t5_finetuning"

job = aiplatform.PipelineJob(
    display_name="t5x-finetuning",
    template_path=f"https://{REGION}-kfp.pkg.dev/{PROJECT_ID}/quickstart-kfp-repo/{templateName}/{versionName}",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
    parameter_values={
        "project_id": PROJECT_ID,
        "accelerator_count": 32,
        "feature_keys": "question",
        "label_key": "answer",
        "training_data_path": "gs://cloud-llm-public/tfds/natural_questions_open/1.0.0_shortened/natural_questions_open-train.tfrecord-00000-of-00001",
        "validation_data_path": "gs://cloud-llm-public/tfds/natural_questions_open/1.0.0_shortened/natural_questions_open-validation.tfrecord-00000-of-00001",
    },
)

job.run()

查看管道结果

In [ ]:
PROJECT_NUMBER = job.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    artifact = ""
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVALUATION_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        # Check if file exists, 0 is success
        !gsutil -q stat $EXECUTE_OUTPUT
        if _exit_code == 0:
            ! gsutil cat $EXECUTE_OUTPUT
            artifact = EXECUTE_OUTPUT
            break
        !gsutil -q stat $GCP_RESOURCES
        if _exit_code == 0:
            ! gsutil cat $GCP_RESOURCES
            artifact = GCP_RESOURCES
            break
        !gsutil -q stat $EVALUATION_METRICS
        if _exit_code == 0:
            ! gsutil cat $EVALUATION_METRICS
            artifact = EVALUATION_METRICS
            break

    return artifact


print("model-upload")
artifacts = print_pipeline_output(job, "model-upload")
output = !gsutil cat $artifacts
print(output)
output = json.loads(output[0])
model_id = output["artifacts"]["model"]["artifacts"][0]["metadata"]["resourceName"]
print("\n\n")

### 删除流水线作业

方法 'delete()' 将会删除流水线作业。

In [ ]:
job.delete()

### 部署模型

接下来，您将模型部署到一个端点：

- 使用从流水线资产中获得的 `model_id` 来实例化一个 Vertex AI 模型资源实例。
- 将 Vertex AI 模型资源部署到一个 Vertex AI 端点资源中。

In [ ]:
model = aiplatform.Model(model_id)
endpoint = model.deploy(
    accelerator_count=1,
    accelerator_type=aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4.name,
    machine_type="n1-standard-4",
)
print(endpoint)

做一个预测

最后，您可以使用部署的模型进行预测。

In [ ]:
endpoint.predict(["this is a test"])

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

endpoint.undeploy_all()
endpoint.delete()
model.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

! rm -rf custom custom.tar.gz

! gcloud artifacts repositories delete $REPO_NAME --project {PROJECT_ID} --location {REGION} --quiet